# Example notebook using South African data

## Download the South African data

We start by running our shell script.  Before running this, be sure to have installed and activated the correct environment.  Run `get_SA_data.sh`:

In [1]:
!./get_SA_data.sh

+ _MY_SCRIPT=./get_SA_data.sh
+++ dirname ./get_SA_data.sh
++ cd .
++ pwd
+ BASEDIR='/Users/aj_yoco/DataScience/Udacity Data Engineering/Geospatial-Lookup-Data-South-Africa'
++ uname -s
+ _UNAME_OUT=Darwin
+ DATA='/Users/aj_yoco/DataScience/Udacity Data Engineering/Geospatial-Lookup-Data-South-Africa/data'
+ mkdir /Users/aj_yoco/DataScience/Udacity Data Engineering/Geospatial-Lookup-Data-South-Africa/data
mkdir: /Users/aj_yoco/DataScience/Udacity: File exists
mkdir: Data: File exists
mkdir: Engineering/Geospatial-Lookup-Data-South-Africa: No such file or directory
+ wget -nc -O '/Users/aj_yoco/DataScience/Udacity Data Engineering/Geospatial-Lookup-Data-South-Africa/data/mdb2016.zip' https://www.arcgis.com/sharing/rest/content/items/cfddb54aab5f4d62b2144d80d49b3fdb/data
File ‘/Users/aj_yoco/DataScience/Udacity Data Engineering/Geospatial-Lookup-Data-South-Africa/data/mdb2016.zip’ already there; not retrieving.
+ '[' '!' -d '/Users/aj_yoco/DataScience/Udacity Data Engineering/Geospatial-

With the data loaded into the **data** folder in the working directory, we're ready to start the python code:

In [2]:
from main import *

We load the raw data: 

In [3]:
geometries, geonames, postal_codes = load_raw_data()
geometries.head(7)

Reading geonames dataset
Reading postal codes
Reading wards
All files read!


,ProvinceCode,ProvinceName,LocalMunicipalityCode,WardNumber,WardID,LocalMunicipalityName,DistrictMunicipalityCode,DistrictMunicipalityName,Year,Shape_Length,Shape_Area,geometry
0,LIM,Limpopo,LIM344,1,93404001,Makhado,DC34,Vhembe,2016,0.253589,0.001676,(POLYGON ((30.08821708200003 -23.1714339829999...
1,LIM,Limpopo,LIM344,2,93404002,Makhado,DC34,Vhembe,2016,0.159002,0.000912,(POLYGON ((30.06416801000006 -23.1493073369999...
2,LIM,Limpopo,LIM344,3,93404003,Makhado,DC34,Vhembe,2016,0.374324,0.003885,(POLYGON ((30.34665565200004 -23.0566725199999...
3,LIM,Limpopo,LIM344,4,93404004,Makhado,DC34,Vhembe,2016,0.206612,0.001345,(POLYGON ((30.40710694300003 -23.0880489339999...
4,LIM,Limpopo,LIM344,5,93404005,Makhado,DC34,Vhembe,2016,0.581000,0.009428,(POLYGON ((30.05988895800004 -23.1745324649999...
5,LIM,Limpopo,LIM344,6,93404006,Makhado,DC34,Vhembe,2016,0.174051,0.001108,(POLYGON ((29.79551943700005 -23.0661248259999...
6,LIM,Limpopo,LIM344,7,93404007,Makhado,DC34,Vhembe,2016,0.638269,0.010192,(POLYGON ((29.91194394200005 -23.0058090589999...


Our methodology involves generating a grid that covers the area of interest.  To that end, South Africa is bounded by latitudes in the range [-35,-22] and longitudes in the range [16,33].  We therefore generate the grid using those to sets with the command 

In [4]:
points_grid = gh.generate_grid(lats=[-35,-22], longs=[16,33], accuracy_m=1000, verbose=True)
points_grid.head(7)

Generating point grid
.................collecting results

Grid of size (2210000, 2) generated!


,latitude,longitude
0,-35.00,16.0
1,-34.99,16.0
2,-34.98,16.0
3,-34.97,16.0
4,-34.96,16.0
5,-34.95,16.0
6,-34.94,16.0


> whenever you see the `gh.`, note that those are functions from the [geohelpers.py](geohelpers.py) file, which you can import and use in your own **main** code with `import geohelpers as gh`

Next, we look up the grid against the geometries - this is the crux of this repository.

In [5]:
located_grid = gh.process_dataframe(points_grid, geometries, accuracy_m=1000, verbose=True)
located_grid.head(7)

Generating the geokey
Generating the geometry points from the coordinates
selected chunksize 44200..................................................Done!
Locating the points
There are many points to locate - this is going to take a while!
.......... (442000 of 2210000 [20%] processed)
.......... (884000 of 2210000 [40%] processed)
.......... (1326000 of 2210000 [60%] processed)
.......... (1768000 of 2210000 [80%] processed)
.......... (2210000 of 2210000 [100%] processed)
Combining results
Done!
1131978 of 2210000 located within geometries
1078022 points were not found within provided geometries!


,latitude,longitude,geokey,geometry,index_geometries,ProvinceCode,ProvinceName,LocalMunicipalityCode,WardNumber,WardID,LocalMunicipalityName,DistrictMunicipalityCode,DistrictMunicipalityName,Year,Shape_Length,Shape_Area
0,-28.63,16.46,-2863000;1646000,POINT (16.46 -28.63),1633,NC,Northern Cape,NC061,2,30601002,Richtersveld,DC6,Namakwa,2016,2.937021,0.210782
1,-28.62,16.46,-2862000;1646000,POINT (16.46 -28.62),1633,NC,Northern Cape,NC061,2,30601002,Richtersveld,DC6,Namakwa,2016,2.937021,0.210782
2,-28.61,16.46,-2861000;1646000,POINT (16.46 -28.61),1633,NC,Northern Cape,NC061,2,30601002,Richtersveld,DC6,Namakwa,2016,2.937021,0.210782
3,-28.60,16.46,-2860000;1646000,POINT (16.46 -28.6),1633,NC,Northern Cape,NC061,2,30601002,Richtersveld,DC6,Namakwa,2016,2.937021,0.210782
4,-28.59,16.46,-2859000;1646000,POINT (16.46 -28.59),1633,NC,Northern Cape,NC061,2,30601002,Richtersveld,DC6,Namakwa,2016,2.937021,0.210782
5,-28.64,16.47,-2864000;1647000,POINT (16.47 -28.64),1633,NC,Northern Cape,NC061,2,30601002,Richtersveld,DC6,Namakwa,2016,2.937021,0.210782
6,-28.63,16.47,-2863000;1647000,POINT (16.47 -28.63),1633,NC,Northern Cape,NC061,2,30601002,Richtersveld,DC6,Namakwa,2016,2.937021,0.210782


> Note that the **geometries** dataframe contains a column named _geometry_ that is a [shapely](https://shapely.readthedocs.io/en/stable/manual.html) geometry Polygon and the grid contains a [shapely](https://shapely.readthedocs.io/en/stable/manual.html) geometry Point.  This allows us to find the geometry containing the point.

With the lookup done, all that remains is decomposing the parts for use in a database.  You want to store an identifier to the geometries file (in our case **ward_id**) in the generated grid file, along with the lookup key.  This becomes the **grid** dataset.  In our case we have this **grid** table with four columns (`ward_id`,`geokey`,`latitude`,`longitude`) and N rows, where N depends on the chosen accuracy_m level.

Next, we save the data by using the `gh.save_data` helper function.  It accepts a dictionary which will be used to filter and rename the columns to keep only the relevant ones.  We also decompose the table into two parts: the **grid** and the **wards** datasets and save each.

In [6]:
grid_cols = {
    'WardID':'ward_id',
    'WardNumber':'ward_number',
    'Shape_Length':'ward_length',
    'Shape_Area':'ward_area',
    'LocalMunicipalityName':'local_municipality',
    'DistrictMunicipalityCode':'district_minicipal_code',
    'DistrictMunicipalityName':'district_municipality',
    'ProvinceName':'province_code',
    'ProvinceCode':'province_name',
    'geokey':'geokey',
    'latitude':'latitude',
    'longitude':'longitude'}
grid = gh.save_data(located_grid, 'located_grid.json.gz','processed_data',columns=grid_cols)
wards = grid.drop(columns=['geokey','latitude','longitude']).drop_duplicates()
grid = grid[['geokey','ward_id','latitude','longitude']].drop_duplicates()

Directory processed_data already exists
Writing file at processed_data/located_grid.json.gz


We save the data we just generated using the `save_data` helper function:

In [7]:
wards = gh.save_data(df=wards, filename='wards.json.gz', directory ='datasets')
wards.head(7)

Directory datasets already exists
Writing file at datasets/wards.json.gz


,ward_id,ward_number,ward_length,ward_area,local_municipality,district_minicipal_code,district_municipality,province_code,province_name
0,30601002,2,2.937021,0.210782,Richtersveld,DC6,Namakwa,Northern Cape,NC
2107,30601001,1,3.745124,0.506391,Richtersveld,DC6,Namakwa,Northern Cape,NC
3455,30601003,3,2.232647,0.137689,Richtersveld,DC6,Namakwa,Northern Cape,NC
3618,30601004,4,0.902299,0.032835,Richtersveld,DC6,Namakwa,Northern Cape,NC
3809,30602008,8,3.165461,0.321461,Nama Khoi,DC6,Namakwa,Northern Cape,NC
9929,30604001,1,3.370907,0.301481,Kamiesberg,DC6,Namakwa,Northern Cape,NC
10347,30604002,2,4.155521,0.441947,Kamiesberg,DC6,Namakwa,Northern Cape,NC


In [8]:
grid = gh.save_data(df=grid, filename='grid.json.gz', directory ='datasets')
grid.head(7)

Directory datasets already exists
Writing file at datasets/grid.json.gz


,geokey,ward_id,latitude,longitude
0,-2863000;1646000,30601002,-28.63,16.46
1,-2862000;1646000,30601002,-28.62,16.46
2,-2861000;1646000,30601002,-28.61,16.46
3,-2860000;1646000,30601002,-28.60,16.46
4,-2859000;1646000,30601002,-28.59,16.46
5,-2864000;1647000,30601002,-28.64,16.47
6,-2863000;1647000,30601002,-28.63,16.47


And that concludes our first section - loading, transforming and saving the data.
We've seen how to generate and store the datasets, now let's look into how to use them to look up incoming points in our warehouse/database.

## Using the stored datasets to look up incoming points

In [9]:
from main import *

We start by generating some sample data from around the Cape Town area.

In [10]:
# generate the input data (on a higher level of accuracy - not unlike real-world location data coming in)
incoming_data = gh.generate_grid(lats=[-34.5,-33.5], longs=[18,19], accuracy_m=10).sample(500)
print(f'Size of sample: {incoming_data.shape}')
incoming_data.head(7)

Generating point grid

Grid of size (100000000, 2) generated!
Size of sample: (500, 2)


,latitude,longitude
50533414,-33.9686,18.5034
43005927,-34.4973,18.4359
92159766,-34.3434,18.9297
73328251,-34.1749,18.7382
57230428,-34.2672,18.5704
91606695,-33.8905,18.9166
34766788,-33.7312,18.3467


Then we transform the latitudes and longitudes into the **geokey** using the accuracy level of our grid data (i.e. 1000m) 

In [11]:
geokey = gh.generate_key(incoming_data, accuracy_m=1000)['geokey']
incoming_data['geokey'] = geokey
incoming_data.head(7)

Generating the geokey


,latitude,longitude,geokey
50533414,-33.9686,18.5034,-3397000;1850000
43005927,-34.4973,18.4359,-3450000;1844000
92159766,-34.3434,18.9297,-3434000;1893000
73328251,-34.1749,18.7382,-3417000;1873999
57230428,-34.2672,18.5704,-3427000;1857000
91606695,-33.8905,18.9166,-3389000;1892000
34766788,-33.7312,18.3467,-3372999;1835000


In SQL, this would look as follows:
```sql
SELECT latitude, longitude, 
    cast(cast(round(latitude,3) as int) as str) + ';'+
    cast(cast(round(longitude,3) as int) as str) as geokey
FROM incoming_data
```

Next, we load the pre-processed data

In [12]:
grid = pd.read_json('datasets/grid.json.gz')
grid.sample(7)

,geokey,ward_id,latitude,longitude
562582,-3319000;2524000,21002006,-33.19,25.24
617288,-2730000;2583000,64004010,-27.30,25.83
661284,-2877000;2625000,41801002,-28.77,26.25
1112680,-2465000;3191000,83205030,-24.65,31.91
1027647,-2499000;3049000,83201013,-24.99,30.49
544767,-2998000;2515000,41602005,-29.98,25.15
927887,-2761000;2953000,41905008,-27.61,29.53


In [13]:
wards = pd.read_json('datasets/wards.json.gz')
wards.sample(7)

,ward_id,ward_number,ward_length,ward_area,local_municipality,district_minicipal_code,district_municipality,province_code,province_name
1887,79900061,61,0.213964,0.002011,City of Tshwane,TSH,City of Tshwane,Gauteng,GT
3309,93403001,1,0.612856,0.008375,Thulamela,DC34,Vhembe,Limpopo,LIM
2459,21505032,32,0.735268,0.006796,Nyandeni,DC15,O.R.Tambo,Eastern Cape,EC
1128,21209002,2,0.261703,0.002117,Raymond Mhlaba,DC12,Amathole,Eastern Cape,EC
1430,63702032,32,0.437306,0.005329,Local Municipality *,DC37,Bojanala,North West,NW
2287,83105032,32,1.154289,0.037494,Thembisile,DC31,Nkangala,Mpumalanga,MP
26,30604004,4,3.977891,0.560068,Kamiesberg,DC6,Namakwa,Northern Cape,NC


To do the "lookup", there are two steps.  Step 1, we simply join the incoming data to the **grid** table using `geokey`

In [14]:
located = incoming_data.merge(grid, how='left', on='geokey')
located.head(7)

,latitude_x,longitude_x,geokey,ward_id,latitude_y,longitude_y
0,-33.9686,18.5034,-3397000;1850000,19100060.0,-33.97,18.50
1,-34.4973,18.4359,-3450000;1844000,NaN,NaN,NaN
2,-34.3434,18.9297,-3434000;1893000,10302010.0,-34.34,18.93
3,-34.1749,18.7382,-3417000;1873999,NaN,NaN,NaN
4,-34.2672,18.5704,-3427000;1857000,NaN,NaN,NaN
5,-33.8905,18.9166,-3389000;1892000,10204005.0,-33.89,18.92
6,-33.7312,18.3467,-3372999;1835000,NaN,NaN,NaN


And then, Step 2, join the result of the above to the **wards** table using `ward_id`, i.e.

In [15]:
final = located.merge(wards, how='left', on='ward_id')
final.head(7)

,latitude_x,longitude_x,geokey,ward_id,latitude_y,longitude_y,ward_number,ward_length,ward_area,local_municipality,district_minicipal_code,district_municipality,province_code,province_name
0,-33.9686,18.5034,-3397000;1850000,19100060.0,-33.97,18.50,60.0,0.183140,0.000761,City of Cape Town,CPT,City of Cape Town,Western Cape,WC
1,-34.4973,18.4359,-3450000;1844000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-34.3434,18.9297,-3434000;1893000,10302010.0,-34.34,18.93,10.0,1.142659,0.020526,Overstrand,DC3,Overberg,Western Cape,WC
3,-34.1749,18.7382,-3417000;1873999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,-34.2672,18.5704,-3427000;1857000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,-33.8905,18.9166,-3389000;1892000,10204005.0,-33.89,18.92,5.0,0.695273,0.010691,Stellenbosch,DC2,Cape Winelands,Western Cape,WC
6,-33.7312,18.3467,-3372999;1835000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


The SQL code for these joins above translate to:
```sql
with located as (
    SELECT 
        i.latitude as latitude_x,
        i.longitude as longitude_x, 
        i.geokey, 
        g.ward_id, 
        g.latitude as latitude_y,
        g.longitude as longitude_y
    FROM incoming_data as i
    LEFT JOIN grid as g on g.geokey=i.geokey)
SELECT l.*, w.*
FROM located as l
LEFT JOIN wards as w on w.ward_id=l.ward_id
```